## RAG: Retrieval-Augmented Generation

<a href="https://colab.research.google.com/github/adithya-s-k/AI-Engineering.academy/blob/main/RAG/00_RAG_from_Scratch/RAG_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



- **R**: **Retrieval** - Fetch the right external content.
- **A**: **Augmentation** - Modify the prompt to pass content form Retrival Stage
- **G**: **Generation** - Generate the final response using the LLM.

### Programmatic Stages:

#### 1. Data Ingestion:
   - Parse PDF and extract text.
   - Perform text chunking.
   - Set up the database.
   - Populate the database with parsed data.

#### 2. Retrieval:
   - Take the user query as input.
   - Perform similarity search across the stored data.
   - Retrieve the most relevant chunks of information.
   
#### 3. Augmentation:
   - Augment the prompt by incorporating relevant chunks of retrieved data.
   - Adjust the prompt through prompt engineering to optimize for clarity and context.

#### 4. Generation:
   - Use the enhanced prompt to generate a response using the LLM.

### Data Ingestion

Load Data

In [1]:
!pip install -q sentence-transformers
!pip install -q wikipedia-api
!pip install -q numpy
!pip install -q scipy
!pip install openai
!pip install rich
!pip install pypdf2
!pip install gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: hug

In [2]:

!pip install python-dotenv

In [3]:
import re
import os
import openai
from rich import print
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import numpy as np
import textwrap
from wikipediaapi import Wikipedia
import PyPDF2

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
def load_document(file_path):
    """
    Load document from a given file path. Supports PDF and text files.
    """
    _, file_extension = os.path.splitext(file_path)

    if file_extension.lower() == '.pdf':
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
    elif file_extension.lower() == '.txt':
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    elif file_extension.lower() == '.md':
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    else:
        raise ValueError("Unsupported file format. Please provide a PDF or text file.")

    return text

data = load_document("/content/a-necessary-evil_-necromancy-and-christian-death.pdf.pdf")

In [5]:
print(data)

Necromancy, the art or practice of magically conjuring up t he souls of the dead, is  
 primarily a form of divination. More generally, ne cromancy is often considered  
 synonymous with black magic, sorcery or witchcraft, perhaps because the calling up  
 of the dead may occur for purposes other than information seeking, or  because the  
 separation of divination from its consequences is not always clear. The re is also a  
 linguistic basis for the expanded use of the word: the term ​ black art ​  for magic appears  
 to be based on a corruption of ​ necromancy ​ (from Greek ​ necros ​ , 'dead') to ​ nigramancy  
 (from Latin ​ niger ​ , 'black'). (Jones, 6451).  
  
 What fascinates me about this term "necromancy" is the process by whi ch it becomes a  
 synonym for other magical practices and is then carefully differentiated from them again. The  
 occult scholarship of the seventeenth century and onwa rds mentions witchcraft and  
 necromancy in almost the same breath, anxious to differentiate the one from the other. Why?  
 If necromancy is synonymous with witchcraft, surely use of the two terms is redunda nt?    
  
 The answer, I discovered, is that they were never synonym ous in the first place. The re is a  
 conflation of terms at work here, but it is between necromancy - divination through di scourse  
 with the deceased - and a form of black magic, concerned with discourse and ba rgains made  
 with demons, devils and other diabolical entities.  
  
 In about 1010, Burchard of Worms' ​ Corrector and Doctor ​  differentiated between  
 "magicians", "diviners" - presumably including ne cromancers - witches and the "you"  being  
 addressed by the text, an ordinary person not  identified as any of  these particular practitioners  
 but still perhaps enacting magic. However, Burchard doe s not separate these practitioners by  
 their practices; for example, his list of divinations includes the influencing of  minds and the  
 weather. John Shinners points out that "Burchard culled most of the sins and their penances  
 in his collection from older penitentials, some dating ba ck to the seventh century" (459): this  
 distinction between practitioners and lack of similar distinction be tween practices appears  
 therefore to be long-established.    
  
 This lack of distinction is even more apparent in the work of Johannes Hartlieb, who ( in the  
 early fifteenth century) defines necromancy thus:  Nygramancia ​ is the first forbidden art, and is called the 
black art. Thi s art is the worst  
 of all, because it proceeds with sacrifices and services that must be rendered to the  
 devils. One who wishes to exerciese this art must give all sorts of sacrifices to the  
 devils, and must take an oath and pact with the devils (in Kieckhefer, 33).  
  
 There is no reference to the dead whatsoever here, sugge sting that necromancy and di abolical  
 sorcery have become fully conflated by the early fifteenth century. Ot her texts allow the  
 process to be tracked more clearly. Isidore of Seville's ​ Etymologies, ​ from the early seventh  
 century, ​  ​ mentions among the forms of divination "the ​ necromantici ​ who resuscitate and  
 interrogate the dead" (Kickehefer, 33). This definition is preserved by Hugh of S t. Victor in  
 the ​ Didascalicon ​  of the early twelfth century, but with the addition of  the diabolical element  
 and the positioning of necromancy as first among di vinations which are apparent in Ha rtlieb:  
  
 primam, necromantiam, quod interpretatur divinatio in mortuis, necros enim Graece,  
 mortuus Latine, unde necromantia, divinatio, qua e fit per sacrificium sanguinis  
 humani, quem daemones sitiunt, et in eo delectantur effuso (810D) .  
  
 I shan't embarrass myself and my schoolboy La tin further by a ttempting a complete  
 translation, but the phrasing "first, necromancy" and its definition as divination through t he  
 dead are apparent even to me, as is the reference to a sacrifice of 

Perform Chunking

In [6]:
def chunk_text(text, chunk_size=100, overlap=20):
    """
    Split the text into chunks based on the number of words and word overlap.
    """
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

chunked_data = chunk_text(data)

chunked_data

['Necromancy, the art or practice of magically conjuring up t he souls of the dead, is primarily a form of divination. More generally, ne cromancy is often considered synonymous with black magic, sorcery or witchcraft, perhaps because the calling up of the dead may occur for purposes other than information seeking, or because the separation of divination from its consequences is not always clear. The re is also a linguistic basis for the expanded use of the word: the term \u200b black art \u200b for magic appears to be based on a corruption of \u200b necromancy \u200b (from Greek \u200b',
 '\u200b black art \u200b for magic appears to be based on a corruption of \u200b necromancy \u200b (from Greek \u200b necros \u200b , \'dead\') to \u200b nigramancy (from Latin \u200b niger \u200b , \'black\'). (Jones, 6451). What fascinates me about this term "necromancy" is the process by whi ch it becomes a synonym for other magical practices and is then carefully differentiated from them again. T

Visualise Chunking

In [7]:
# Print the list of chunks
def print_chunks(chunks):
    for i, chunk in enumerate(chunks):
        print(f"Chunk {i + 1}:")
        print(chunk)
        print("-" * 50)

print_chunks(chunked_data)

Chunk 1:

Necromancy, the art or practice of magically conjuring up t he souls of the dead, is primarily a form of 
divination. More generally, ne cromancy is often considered synonymous with black magic, sorcery or witchcraft, 
perhaps because the calling up of the dead may occur for purposes other than information seeking, or because the 
separation of divination from its consequences is not always clear. The re is also a linguistic basis for the 
expanded use of the word: the term ​ black art ​ for magic appears to be based on a corruption of ​ necromancy ​ (from
Greek ​

--------------------------------------------------

Chunk 2:

​ black art ​ for magic appears to be based on a corruption of ​ necromancy ​ (from Greek ​ necros ​ , 'dead') to ​
nigramancy (from Latin ​ niger ​ , 'black'). (Jones, 6451). What fascinates me about this term "necromancy" is the 
process by whi ch it becomes a synonym for other magical practices and is then carefully differentiated from them 
again. The occult scholarship of the seventeenth century and onwa rds mentions witchcraft and necromancy in almost 
the same breath, anxious to differentiate the one from the other. Why? If necromancy is synonymous with witchcraft,
surely

...

Religion ​ , volume 10, second edition. (Farmington MI : Macmillan, 2005). Kieckhefer, Richard. ​ Forbidden Rites: A
Necromancer's Manual of the Fifteenth Century (Pennsylvania: Pennsylvania State University Press, 1998) . Le Goff, 
Jacques. ​ The Birth of Purgatory ​ , trans. Arthur Goldhammer. (Aldershot: Scolar Press, 1984 [1981]). Lee, 
Frederick George (editor). ​ Glimpses of the Supernatural ​ vol. 1 (London : Henry S. King & co., 1875). Rosack, C.
W. ​ The Mysteries of Astrology and the Wonde rs of Magic including a Hi story of the Rise and Progress of Astrology
and the various branches of Necromanc

--------------------------------------------------

Chunk 41:

rs of Magic including a Hi story of the Rise and Progress of Astrology and the various branches of Necromanc y - 
and so on ​ . (London: Sampson, Low, Son & Company, 1854) . Shinners, John. ​ Medieval Popular Religion 1000-1500: A
Reader ​ . Second edition. (Toronto: Broadview Press, 2007). Waite, A.E.. ​ The Book of Black Magic and of Pacts ​ .
(London: George Redway, 1898 ).

--------------------------------------------------

Setting Up embedding model

In [9]:
# Load the sentence transformer model for embeddings
model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling.py:   0%|          | 0.00/59.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

set up similarity function

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

visualise embeddings

In [32]:
def generate_embeddings(text_chunks):
    # Assuming you have a function or model to get embeddings
    return [model.encode(chunk).tolist() for chunk in text_chunks]
embeddings = generate_embeddings(chunked_data)

embed chunks

In [ ]:
#visualize_embeddings(chunked_data, model)

store the vectors/embedding

In [16]:
!pip install -U qdrant_client fastembed


  Using cached protobuf-5.28.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 19.6 MB/s eta 0:00:00
Using cached protobuf-5.28.3-cp38-abi3-manylinux2014_x86_64.whl (316 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1
ERROR: pip's dependency resolver does not curren

## Retrival

set up vector store

In [22]:
from qdrant_client import QdrantClient
client = QdrantClient(path="./db/")

In [24]:
inp_collection_name = "necromancy"
from qdrant_client import models

client.create_collection(
    collection_name="my-inp_collection_name",
    vectors_config={
        "embedding": models.VectorParams(
            size=768,  # Adjust based on your embedding size
            distance=models.Distance.COSINE,
        )
    }
)

True

In [40]:
points = []
#print(embeddings)
for id, (embedding) in enumerate(zip(embeddings)):
    points.append(models.PointStruct(
        id=id,
        # Pass the embedding as a dictionary with the field name "embedding"
        vector={"embedding": embedding},
    ))

In [41]:
batch_size = 100  # Adjust based on your needs
for i in range(0, len(points), batch_size):
    batch = points[i:i + batch_size]
    client.upload_points("my-inp_collection_name", points=batch, wait=True)

In [43]:
print(f"Total points in collection: {client.count('my-inp_collection_name').count}")

Total points in collection: 41

similarity search

get top K results

## Augmentation

Augmenting Prompt

modifying system prompt

## Generation

set up llm provider

generate response